In [1]:
import warnings
warnings.filterwarnings("ignore") # suppress h5py deprecation warning

import os
import backtrader as bt
import numpy as np

from btgym import BTgymEnv, BTgymDataset
from btgym.strategy.observers import Reward, Position, NormPnL
from btgym.algorithms import Launcher, Unreal, AacStackedRL2Policy
from btgym.research import DevStrat_4_11

### Stacked LSTM Agent usage example.

Based on NAV_A3C+D from ["LEARNING TO NAVIGATE IN COMPLEX ENVIRONMENTS"](https://arxiv.org/pdf/1611.03673.pdf) paper by Mirowski at al.;

Modifications to original paper arhcitecture:
- splitted Policy/Value outputs: Policy is taken off first LSTM layer, Value - off the second;
- LSTM state initialisation: first RNN layer context (policy) is initialised on every episode start, while second   (Value) is reset either on begining of every Trial (future work) or or every N-constant episodes (60 for this     example), motivated by RL^2 approach by Duan et al., 
  ["FAST REINFORCEMENT LEARNING VIA SLOW REINFORCEMENT LEARNING"](https://arxiv.org/pdf/1611.02779.pdf);
- inner/external observation state state split: external (market) is encoded via conolution layers and fed to       first LSTM layer, inner (broker) state is fed into second LSTM layer, can optionally be encoded via separate       convolution block (doesnt seem to improve much though);
- optional Value Replay losss (`Unreal` feature) improves sample efficiency, but is computationally expensive;

Other details:
- All convolution and LSTM layers are layer-normalized, see 
  ["Layer Normalisation"](https://arxiv.org/abs/1607.06450) paper by Jimmy Ba at al.;
  
- Upd 2.02.18: linear layers aer Noisy-Net ones, see: [Noisy Networks for Exploration] (https://arxiv.org/abs/1706.10295) paper by Fortunato at al.; policy output is centered using layer normalisation;
 added linearly decayed state scaling;

- A3C option `time_flat` is ON by default, improves training stability, reduces computation costs, see 
  [Base_AAC class Note](https://kismuz.github.io/btgym/btgym.algorithms.html#module-btgym.algorithms.aac) for       details;
  
Diagram: https://kismuz.github.io/btgym/_images/a3c_stacked_lstm_agent.png

**NOTE:**
Currently it takes ~20M env.steps to fit 6-month 1min bars data set. Much faster on smaller ones.

In [2]:
# Set backtesting engine parameters:

MyCerebro = bt.Cerebro()

MyCerebro.addstrategy(
    DevStrat_4_11,
    drawdown_call=10, # max % to loose, in percent of initial cash
    target_call=10,  # max % to win, same
    skip_frame=10,
    gamma=0.99,
    reward_scale=7, # gardient`s nitrox, touch with care!
    state_ext_scale = np.linspace(3e3, 1e3, num=5)
)
# Set leveraged account:
MyCerebro.broker.setcash(2000)
MyCerebro.broker.setcommission(commission=0.0001, leverage=10.0) # commisssion to imitate spread
MyCerebro.addsizer(bt.sizers.SizerFix, stake=5000,)  

# Visualisations for reward, position and PnL dynamics:
MyCerebro.addobserver(Reward)
MyCerebro.addobserver(Position)
MyCerebro.addobserver(NormPnL)

# Data: uncomment to get up to six month of 1 minute bars:
data_m1_6_month = [
    './data/DAT_ASCII_EURUSD_M1_201701.csv',
    './data/DAT_ASCII_EURUSD_M1_201702.csv',
    './data/DAT_ASCII_EURUSD_M1_201703.csv',
    './data/DAT_ASCII_EURUSD_M1_201704.csv',
    #'./data/DAT_ASCII_EURUSD_M1_201705.csv',
    #'./data/DAT_ASCII_EURUSD_M1_201706.csv',
]

# Uncomment single choice:
MyDataset = BTgymDataset(
    #filename=data_m1_6_month,
    filename='./data/test_sine_1min_period256_delta0002.csv',  # simple sine 
    start_weekdays={0, 1, 2, 3, 4, 5, 6},
    episode_duration={'days': 1, 'hours': 23, 'minutes': 40}, # note: 2day-long episode
    start_00=False,
    time_gap={'hours': 10},
)

env_config = dict(
    class_ref=BTgymEnv, 
    kwargs=dict(
        dataset=MyDataset,
        engine=MyCerebro,
        render_modes=['episode', 'human', 'external', 'internal'],
        render_state_as_image=True,
        render_ylabel='OHL_diff. / Internals',
        render_size_episode=(12,8),
        render_size_human=(9, 4),
        render_size_state=(11, 3),
        render_dpi=75,
        port=5000,
        data_port=4999,
        connect_timeout=90,
        verbose=0,
    )
)

cluster_config = dict(
    host='127.0.0.1',
    port=12230,
    num_workers=1,  # set according CPU's available or so
    num_ps=1,
    num_envs=1,
    log_dir=os.path.expanduser('~/tmp/test_4_11'),
)

policy_config = dict(
    class_ref=AacStackedRL2Policy,
    kwargs={
        'lstm_layers': (256, 256),
        'lstm_2_init_period': 60,
    }
)

trainer_config = dict(
    class_ref=Unreal,
    kwargs=dict(
        opt_learn_rate=[1e-4, 1e-4], # random log-uniform 
        opt_end_learn_rate=1e-5,
        opt_decay_steps=50*10**6,
        model_gamma=0.99,
        model_gae_lambda=1.0,
        model_beta=0.01, # entropy reg
        rollout_length=20,
        time_flat=True, 
        use_value_replay=False, 
        model_summary_freq=100,
        episode_summary_freq=5,
        env_render_freq=20,
    )
)

In [4]:
launcher = Launcher(
    cluster_config=cluster_config,
    env_config=env_config,
    trainer_config=trainer_config,
    policy_config=policy_config,
    test_mode=False,
    max_env_steps=100*10**6,
    root_random_seed=0,
    purge_previous=1,  # ask to override previously saved model and logs
    verbose=0
)

# Train it:
launcher.run()

</Users/muzikin/tmp/test_4_11> already exists. Override[y/n]? y
[2018-07-10 14:39:25.078327] NOTICE: LauncherShell: Files in </Users/muzikin/tmp/test_4_11> purged.
[2018-07-10 14:39:27.454439] NOTICE: UNREAL_0: learn_rate: 0.000100, entropy_beta: 0.010000

********************************************************************************************
**  Press `Ctrl-C` or jupyter:[Kernel]->[Interrupt] to stop training and close launcher.  **
********************************************************************************************

Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path /Users/muzikin/tmp/test_4_11/train/model.ckpt
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:global/global_step/sec: 0
[2018-07-10 14:39:41.185906] NOTICE: Worker_0: started training at step: 0
[2018-07-10 1

RUNNER: one_hot: [1. 0. 0. 0.], vec: [0], dict: OrderedDict([('base_data_line', 0)])
step:  OrderedDict([('base_data_line', 0)]) {'base_data_line': 'hold'}
RUNNER: one_hot: [1. 0. 0. 0.], vec: [0], dict: OrderedDict([('base_data_line', 0)])
step:  OrderedDict([('base_data_line', 0)]) {'base_data_line': 'hold'}
RUNNER: one_hot: [1. 0. 0. 0.], vec: [0], dict: OrderedDict([('base_data_line', 0)])
step:  OrderedDict([('base_data_line', 3)]) {'base_data_line': 'close'}
RUNNER: one_hot: [0. 0. 0. 1.], vec: [3], dict: OrderedDict([('base_data_line', 3)])
step:  OrderedDict([('base_data_line', 0)]) {'base_data_line': 'hold'}
RUNNER: one_hot: [1. 0. 0. 0.], vec: [0], dict: OrderedDict([('base_data_line', 0)])
step:  OrderedDict([('base_data_line', 0)]) {'base_data_line': 'hold'}
RUNNER: one_hot: [1. 0. 0. 0.], vec: [0], dict: OrderedDict([('base_data_line', 0)])
step:  OrderedDict([('base_data_line', 3)]) {'base_data_line': 'close'}
RUNNER: one_hot: [0. 0. 0. 1.], vec: [3], dict: OrderedDict([(

RUNNER: one_hot: [0. 0. 1. 0.], vec: [2], dict: OrderedDict([('base_data_line', 2)])
step:  OrderedDict([('base_data_line', 2)]) {'base_data_line': 'sell'}
step:  OrderedDict([('base_data_line', 0)]) {'base_data_line': 'hold'}
RUNNER: one_hot: [1. 0. 0. 0.], vec: [0], dict: OrderedDict([('base_data_line', 0)])
step:  OrderedDict([('base_data_line', 0)]) {'base_data_line': 'hold'}
RUNNER: one_hot: [1. 0. 0. 0.], vec: [0], dict: OrderedDict([('base_data_line', 0)])
step:  OrderedDict([('base_data_line', 0)]) {'base_data_line': 'hold'}
RUNNER: one_hot: [1. 0. 0. 0.], vec: [0], dict: OrderedDict([('base_data_line', 0)])
step:  OrderedDict([('base_data_line', 0)]) {'base_data_line': 'hold'}
RUNNER: one_hot: [1. 0. 0. 0.], vec: [0], dict: OrderedDict([('base_data_line', 0)])
step:  OrderedDict([('base_data_line', 0)]) {'base_data_line': 'hold'}
RUNNER: one_hot: [1. 0. 0. 0.], vec: [0], dict: OrderedDict([('base_data_line', 0)])
step:  OrderedDict([('base_data_line', 2)]) {'base_data_line': '

step:  OrderedDict([('base_data_line', 0)]) {'base_data_line': 'hold'}
RUNNER: one_hot: [1. 0. 0. 0.], vec: [0], dict: OrderedDict([('base_data_line', 0)])
step:  OrderedDict([('base_data_line', 0)]) {'base_data_line': 'hold'}
RUNNER: one_hot: [1. 0. 0. 0.], vec: [0], dict: OrderedDict([('base_data_line', 0)])
step:  OrderedDict([('base_data_line', 0)]) {'base_data_line': 'hold'}
RUNNER: one_hot: [1. 0. 0. 0.], vec: [0], dict: OrderedDict([('base_data_line', 0)])
step:  OrderedDict([('base_data_line', 0)]) {'base_data_line': 'hold'}
RUNNER: one_hot: [1. 0. 0. 0.], vec: [0], dict: OrderedDict([('base_data_line', 0)])
step:  OrderedDict([('base_data_line', 2)]) {'base_data_line': 'sell'}
RUNNER: one_hot: [0. 0. 1. 0.], vec: [2], dict: OrderedDict([('base_data_line', 2)])
step:  OrderedDict([('base_data_line', 0)]) {'base_data_line': 'hold'}
RUNNER: one_hot: [1. 0. 0. 0.], vec: [0], dict: OrderedDict([('base_data_line', 0)])
step:  OrderedDict([('base_data_line', 2)]) {'base_data_line': '

RUNNER: one_hot: [0. 1. 0. 0.], vec: [1], dict: OrderedDict([('base_data_line', 1)])
step:  OrderedDict([('base_data_line', 1)]) {'base_data_line': 'buy'}
RUNNER: one_hot: [0. 1. 0. 0.], vec: [1], dict: OrderedDict([('base_data_line', 1)])
step:  OrderedDict([('base_data_line', 1)]) {'base_data_line': 'buy'}
RUNNER: one_hot: [0. 1. 0. 0.], vec: [1], dict: OrderedDict([('base_data_line', 1)])
step:  OrderedDict([('base_data_line', 2)]) {'base_data_line': 'sell'}
RUNNER: one_hot: [0. 0. 1. 0.], vec: [2], dict: OrderedDict([('base_data_line', 2)])
step:  OrderedDict([('base_data_line', 1)]) {'base_data_line': 'buy'}
RUNNER: one_hot: [0. 1. 0. 0.], vec: [1], dict: OrderedDict([('base_data_line', 1)])
step:  OrderedDict([('base_data_line', 2)]) {'base_data_line': 'sell'}
RUNNER: one_hot: [0. 0. 1. 0.], vec: [2], dict: OrderedDict([('base_data_line', 2)])
step:  OrderedDict([('base_data_line', 2)]) {'base_data_line': 'sell'}
RUNNER: one_hot: [0. 0. 1. 0.], vec: [2], dict: OrderedDict([('base

...

Process BTgymDataFeedServer-4:1:
Traceback (most recent call last):
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()


[2018-07-10 14:40:21.127479] NOTICE: LauncherShell: chief_worker_0 has joined.


  File "/Users/muzikin/Yandex.Disk.localized/work/btgym/btgym/dataserver.py", line 135, in run
    service_input = socket.recv_pyobj()


[2018-07-10 14:40:21.129914] NOTICE: LauncherShell: parameter_server_0 has joined.
[2018-07-10 14:40:21.131771] NOTICE: LauncherShell: Launcher closed.


  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/site-packages/zmq/sugar/socket.py", line 491, in recv_pyobj
    msg = self.recv(flags)
  File "zmq/backend/cython/socket.pyx", line 693, in zmq.backend.cython.socket.Socket.recv (zmq/backend/cython/socket.c:7683)
  File "zmq/backend/cython/socket.pyx", line 727, in zmq.backend.cython.socket.Socket.recv (zmq/backend/cython/socket.c:7460)
  File "zmq/backend/cython/socket.pyx", line 145, in zmq.backend.cython.socket._recv_copy (zmq/backend/cython/socket.c:2344)
  File "zmq/backend/cython/checkrc.pxd", line 12, in zmq.backend.cython.checkrc._check_rc (zmq/backend/cython/socket.c:9621)
KeyboardInterrupt
Process DrawCerebro-4:2:7:
Traceback (most recent call last):
  File "/Users/muzikin/anaconda/envs/tensorforce/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/muzikin/Yandex.Disk.localized/work/btgym/btgym/rendering/plotter.py", line 74, in run
    figfilename='_tmp_btgym_render.p